In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:
data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'
train_data = TabularDataset(f'{data_url}train.csv')
train_data.head()

,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [3]:
print(train_data.shape)

(10000, 19)


In [4]:
print(*list(train_data.columns), sep='\n')

Unnamed: 0
chern_simons
cusp_volume
hyperbolic_adjoint_torsion_degree
hyperbolic_torsion_degree
injectivity_radius
longitudinal_translation
meridinal_translation_imag
meridinal_translation_real
short_geodesic_imag_part
short_geodesic_real_part
Symmetry_0
Symmetry_D3
Symmetry_D4
Symmetry_D6
Symmetry_D8
Symmetry_Z/2 + Z/2
volume
signature


In [5]:
label = 'signature'
train_data[label].describe()

count    10000.000000
mean        -0.022000
std          3.025166
min        -12.000000
25%         -2.000000
50%          0.000000
75%          2.000000
max         12.000000
Name: signature, dtype: float64

In [6]:
print('Labels:', sorted(train_data[label].unique()))

Labels: [-12, -10, -8, -6, -4, -2, 0, 2, 4, 6, 8, 10, 12]


In [7]:
import os

PATH_MODEL = os.path.join('models', 'knot_theory')

In [8]:
predictor = TabularPredictor(label=label, path=PATH_MODEL).fit(train_data)

Beginning AutoGluon training ...
AutoGluon will save models to "models/knot_theory/"
AutoGluon Version:  0.8.2
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #167~18.04.1-Ubuntu SMP Wed May 24 00:51:42 UTC 2023
Disk Space Avail:   272.39 GB / 501.38 GB (54.3%)
Train Data Rows:    10000
Train Data Columns: 18
Label Column: signature
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	First 10 (of 13) unique label values:  [-2, 0, 2, -8, 4, -4, -6, 8, 6, 10]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9984
Train Data Class Count: 9
Using Feature Generators to preprocess the dat

In [9]:
test_data = TabularDataset(f'{data_url}test.csv')

y_pred = predictor.predict(test_data.drop(columns=[label]))
y_pred.head()

Loaded data from: https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/test.csv | Columns = 19 / 19 | Rows = 5000 -> 5000


0   -4
1   -2
2    0
3    4
4    2
Name: signature, dtype: int64

In [10]:
# MMC: Matthews Correlation Coefficient
predictor.evaluate(test_data, silent=True)

{'accuracy': 0.949,
 'balanced_accuracy': 0.7533344932443743,
 'mcc': 0.9375134384583639}

In [11]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.9490,0.966967,0.985015,0.193196,25.205996,0.007601,0.000586,0.626124,2,True,14
1,LightGBM,0.9456,0.955956,0.082473,0.014431,2.862373,0.082473,0.014431,2.862373,1,True,5
2,XGBoost,0.9448,0.956957,0.281761,0.036778,4.024323,0.281761,0.036778,4.024323,1,True,11
3,LightGBMLarge,0.9444,0.949950,0.423135,0.057662,7.497983,0.423135,0.057662,7.497983,1,True,13
4,CatBoost,0.9432,0.955956,0.017815,0.004302,23.446050,0.017815,0.004302,23.446050,1,True,8
5,RandomForestEntr,0.9384,0.949950,0.113250,0.054561,1.618758,0.113250,0.054561,1.618758,1,True,7
6,NeuralNetFastAI,0.9374,0.945946,0.054609,0.014090,7.832511,0.054609,0.014090,7.832511,1,True,3
7,ExtraTreesGini,0.9360,0.946947,0.151695,0.054460,0.783327,0.151695,0.054460,0.783327,1,True,9
8,ExtraTreesEntr,0.9358,0.942943,0.187581,0.054564,0.811658,0.187581,0.054564,0.811658,1,True,10
9,RandomForestGini,0.9352,0.944945,0.114073,0.054161,0.947414,0.114073,0.054161,0.947414,1,True,6


In [12]:
predictor.set_model_best

<bound method TabularPredictor.set_model_best of <autogluon.tabular.predictor.predictor.TabularPredictor object at 0x7ff98ea2dc40>>